# Machine Translation Project

### Step 1. Import dependencies and script setup

In [13]:
%load_ext autoreload
%autoreload 1

import os
import collections
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Concatenate, LSTM
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from tensorflow.python.client import device_lib

# Data loader
def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()
    return data.split('\n')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# CPU/GPU device check
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8435159831571808138
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5376753664
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11986777501007342572
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


### Step 2. Data load and exploration

In [18]:
# English sentences
english_sentences = load_data('data/small_vocab_en')
# French sentences
french_sentences = load_data('data/small_vocab_fr')

In [21]:
# Sentence examples
for sample_i in range(2):
    print('English sentence {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('French translation {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

English sentence 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
French translation 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
English sentence 2:  the united states is usually chilly during july , and it is usually freezing in november .
French translation 2:  les Ã©tats-unis est gÃ©nÃ©ralement froid en juillet , et il gÃ¨le habituellement en novembre .


In [23]:
# Data exploration and main metrics

english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


### Step 3. Data preprocessing

#### Tokenization

In [26]:
# Tokenization helper function

def tokenize(x):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


#### Padding

In [28]:
# Padding helper function

def pad(x, length=None):
    if length is None:
        length = max(len(i) for i in x)
    padded_sequences = pad_sequences(x, maxlen = length, padding = 'post')
    
    return padded_sequences

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


#### Preprocessing pipeline

In [30]:
# Preprocessing pipeline helper function

def preprocess(x, y):
  
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)
    
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 345


## Models descriptions in increasing complexity

* Model 1. Simple RNN (GRU)
* Model 2. GRU With embedding
* Model 3. Bidirectional GRU
* Model 4. Encoder-Decoder GRU
* Model 5. Final model (combination of the above)

In [37]:
# IDs to text helper function

def logits_to_text(logits, tokenizer):
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

### Model 1. Simple RNN (GRU)

In [39]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    learning_rate = 1e-3
    
    inputs = Input(shape = input_shape[1:])
    x = GRU(64, return_sequences = True)(inputs)
    x = GRU(64, return_sequences = True)(x)
    x = Dense(128, activation = 'relu')(x)
    outputs = Dense(french_vocab_size, activation = 'softmax')(x)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
862/862 [==============================] - 13s 8ms/step - loss: 1.8083 - accuracy: 0.5695 - val_loss: nan - val_accuracy: 0.6512
Epoch 2/10
862/862 [==============================] - 6s 7ms/step - loss: 1.1370 - accuracy: 0.6684 - val_loss: nan - val_accuracy: 0.6798
Epoch 3/10
862/862 [==============================] - 6s 7ms/step - loss: 1.0159 - accuracy: 0.6884 - val_loss: nan - val_accuracy: 0.7004
Epoch 4/10
862/862 [==============================] - 6s 7ms/step - loss: 0.9296 - accuracy: 0.7120 - val_loss: nan - val_accuracy: 0.7273
Epoch 5/10
862/862 [==============================] - 6s 7ms/step - loss: 0.8700 - accuracy: 0.7285 - val_loss: nan - val_accuracy: 0.7346
Epoch 6/10
862/862 [==============================] - 6s 7ms/step - loss: 0.8278 - accuracy: 0.7393 - val_loss: nan - val_accuracy: 0.7465
Epoch 7/10
862/862 [==============================] - 6s 7ms/step - loss: 0.7899 - accuracy: 0.7496 - val_loss: nan - val_accuracy: 0.7490
Epoch 8/10
862/862 [======

### Model 2. GRU With embedding

In [42]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    learning_rate = 1e-3
    
    inputs = Input(shape = input_shape[1:])
    x = Embedding(input_dim = english_vocab_size, output_dim = 128, input_length = input_shape[1])(inputs)
    x = GRU(64, return_sequences = True)(x)
    x = GRU(64, return_sequences = True)(x)
    x = Dense(128, activation = 'relu')(x)
    outputs = Dense(french_vocab_size, activation = 'softmax')(x)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

# Inputs reshape
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

rnn_model_with_embedding = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
rnn_model_with_embedding.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

# Print predictions
print(logits_to_text(rnn_model_with_embedding.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
862/862 [==============================] - 10s 10ms/step - loss: 1.3628 - accuracy: 0.6837 - val_loss: nan - val_accuracy: 0.8228
Epoch 2/10
862/862 [==============================] - 8s 9ms/step - loss: 0.5117 - accuracy: 0.8477 - val_loss: nan - val_accuracy: 0.8643
Epoch 3/10
862/862 [==============================] - 8s 9ms/step - loss: 0.3895 - accuracy: 0.8782 - val_loss: nan - val_accuracy: 0.8878
Epoch 4/10
862/862 [==============================] - 8s 9ms/step - loss: 0.3303 - accuracy: 0.8945 - val_loss: nan - val_accuracy: 0.9009
Epoch 5/10
862/862 [==============================] - 8s 9ms/step - loss: 0.2945 - accuracy: 0.9043 - val_loss: nan - val_accuracy: 0.9083
Epoch 6/10
862/862 [==============================] - 8s 9ms/step - loss: 0.2692 - accuracy: 0.9113 - val_loss: nan - val_accuracy: 0.9132
Epoch 7/10
862/862 [==============================] - 8s 9ms/step - loss: 0.2519 - accuracy: 0.9160 - val_loss: nan - val_accuracy: 0.9157
Epoch 8/10
862/862 [=====

### Model 3. Bidirectional GRU

In [46]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):

    learning_rate = 1e-3
    
    inputs = Input(shape = input_shape[1:])
    x = Bidirectional(GRU(64, return_sequences = True))(inputs)
    x = Bidirectional(GRU(64, return_sequences = True))(x)
    x = Dense(128, activation = 'relu')(x)
    outputs = Dense(french_vocab_size, activation = 'softmax')(x)
    
    model = Model(inputs = inputs, outputs = outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model

# Inputs reshape
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

bidirectional_rnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

# Train the model
bidirectional_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print predictions
print(logits_to_text(bidirectional_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Epoch 1/10
108/108 [==============================] - 6s 25ms/step - loss: 2.7943 - accuracy: 0.4811 - val_loss: nan - val_accuracy: 0.5430
Epoch 2/10
108/108 [==============================] - 2s 19ms/step - loss: 1.6497 - accuracy: 0.5867 - val_loss: nan - val_accuracy: 0.6174
Epoch 3/10
108/108 [==============================] - 2s 18ms/step - loss: 1.3291 - accuracy: 0.6311 - val_loss: nan - val_accuracy: 0.6447
Epoch 4/10
108/108 [==============================] - 2s 18ms/step - loss: 1.1850 - accuracy: 0.6549 - val_loss: nan - val_accuracy: 0.6631
Epoch 5/10
108/108 [==============================] - 2s 19ms/step - loss: 1.0812 - accuracy: 0.6747 - val_loss: nan - val_accuracy: 0.6831
Epoch 6/10
108/108 [==============================] - 2s 18ms/step - loss: 1.0064 - accuracy: 0.6903 - val_loss: nan - val_accuracy: 0.6943
Epoch 7/10
108/108 [==============================] - 2s 18ms/step - loss: 0.9455 - accuracy: 0.7044 - val_loss: nan - val_accuracy: 0.7100
Epoch 8/10
108/108 [

In [ ]:
!git add .
!git commit -m ""